In [50]:
import pandas as pd

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Data Cleaning
from sklearn.feature_extraction.text import CountVectorizer

# PreProcessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Splitting Data
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score

# Modeling
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import LinearSVC



In [51]:
data = pd.read_csv("Starbucks satisfactory survey.csv")
#change data df_copy


In [52]:
# create a copy of the original data 
df = data.copy()
df = pd.DataFrame(df)


In [53]:
# rename all columns
df.rename(columns={
    'Timestamp': 'timestamp',
    '1. Your Gender': 'gender',
    '2. Your Age': 'age',
    '3. Are you currently....?' : 'job',
    '4. What is your annual income?' : 'income',
    '5. How often do you visit Starbucks?' : 'visit_frequency',
    '6. How do you usually enjoy Starbucks?' : 'visit_method',
    '7. How much time do you normally  spend during your visit?' : 'visit_duration',
    '8. The nearest Starbucks\'s outlet to you is...?' : 'distance',
    '9. Do you have Starbucks membership card?' : 'membership',
    '10. What do you most frequently purchase at Starbucks?' : 'item_list',
    '11. On average, how much would you spend at Starbucks per visit?' : 'visit_amount_spent',
    '12. How would you rate the quality of Starbucks compared to other brands (Coffee Bean, Old Town White Coffee..) to be:': 'brand_rating',
    '13. How would you rate the price range at Starbucks?': 'price_rating',
    '14. How important are sales and promotions in your purchase decision?': 'promo_rating',
    '15. How would you rate the ambiance at Starbucks? (lighting, music, etc...)': 'ambiance_rating',
    '16. You rate the WiFi quality at Starbucks as..': 'wifi_rating',
    '17. How would you rate the service at Starbucks? (Promptness, friendliness, etc..)': 'service_rating',
    '18. How likely you will choose Starbucks for doing business meetings or hangout with friends?': 'purpose_biz',
    '19. How do you come to hear of promotions at Starbucks? Check all that apply.': 'channel_list',
    '20. Will you continue buying at Starbucks?': 'revisit'
}, inplace = True)

df.head(10)
df.info()
df_copy = df.copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122 entries, 0 to 121
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   timestamp           122 non-null    object
 1   gender              122 non-null    object
 2   age                 122 non-null    object
 3   job                 122 non-null    object
 4   income              122 non-null    object
 5   visit_frequency     122 non-null    object
 6   visit_method        121 non-null    object
 7   visit_duration      122 non-null    object
 8   distance            122 non-null    object
 9   membership          122 non-null    object
 10  item_list           122 non-null    object
 11  visit_amount_spent  122 non-null    object
 12  brand_rating        122 non-null    int64 
 13  price_rating        122 non-null    int64 
 14  promo_rating        122 non-null    int64 
 15  ambiance_rating     122 non-null    int64 
 16  wifi_rating         122 no

In [54]:
# delete unnecessary columns
df = df.drop(columns=['timestamp'])



In [55]:
# count missing values
df.isna().sum() / len(df.index)*100

gender                0.000000
age                   0.000000
job                   0.000000
income                0.000000
visit_frequency       0.000000
visit_method          0.819672
visit_duration        0.000000
distance              0.000000
membership            0.000000
item_list             0.000000
visit_amount_spent    0.000000
brand_rating          0.000000
price_rating          0.000000
promo_rating          0.000000
ambiance_rating       0.000000
wifi_rating           0.000000
service_rating        0.000000
purpose_biz           0.000000
channel_list          0.819672
revisit               0.000000
dtype: float64

In [56]:
# check data types for each column
df.dtypes
# two columns need extra care: promo_channel 

gender                object
age                   object
job                   object
income                object
visit_frequency       object
visit_method          object
visit_duration        object
distance              object
membership            object
item_list             object
visit_amount_spent    object
brand_rating           int64
price_rating           int64
promo_rating           int64
ambiance_rating        int64
wifi_rating            int64
service_rating         int64
purpose_biz            int64
channel_list          object
revisit               object
dtype: object

In [57]:
for col in df.columns:
    value_counts = df[col].value_counts()
    print(f"Value counts for column '{col}':")
    print(value_counts.to_string())  
    print()

Value counts for column 'gender':
gender
Female    65
Male      57

Value counts for column 'age':
age
From 20 to 29    85
From 30 to 39    17
Below 20         13
40 and above      7

Value counts for column 'job':
job
Employed         61
Student          42
Self-employed    17
Housewife         2

Value counts for column 'income':
income
Less than RM25,000       71
RM25,000 - RM50,000      25
RM50,000 - RM100,000     17
More than RM150,000       6
RM100,000 - RM150,000     3

Value counts for column 'visit_frequency':
visit_frequency
Rarely     76
Monthly    26
Weekly      9
Never       9
Daily       2

Value counts for column 'visit_method':
visit_method
Take away             49
Dine in               46
Drive-thru            20
never                  2
Never buy              1
I dont like coffee     1
Never                  1
Never                  1

Value counts for column 'visit_duration':
visit_duration
Below 30 minutes                73
Between 30 minutes to 1 hour    34
Between

In [58]:
# extra care for item_list and channel_list columns
# step 1: unique values
print(df['item_list'].unique())

## step 2: user CountVectorizer to tokenize items in the column 'item_list' with ";" as delimiter and strip any white spaces
# create an object of CountVectorizer
vectorizer = CountVectorizer(tokenizer=lambda x: x.split(';'))

# perform some data cleaning and invoke the fit_transform method, which is a function that belongs to the CountVectorizer object I created.
df['item_list'] = df['item_list'].str.strip().str.lower()
item_purchased = vectorizer.fit_transform(df['item_list'])

print(len(vectorizer.get_feature_names_out()))
print(vectorizer.get_feature_names_out())

# step 3: transform item_list into a dataframe
item_purchased_df = pd.DataFrame(item_purchased.toarray(), columns=vectorizer.get_feature_names_out())
item_purchased_df.drop(columns=['never', 'never buy any', 'nothing'], inplace=True)
item_purchased_df.columns = ['item_' + col for col in item_purchased_df.columns]

# step 4: merge item_purchased_df with original df
df = pd.concat([df, item_purchased_df], axis=1)

df.head()



['Coffee' 'Cold drinks;Pastries' 'Coffee;Sandwiches' 'Cold drinks'
 'Coffee;Cold drinks' 'Cold drinks;Pastries;Sandwiches'
 'Coffee;Juices;Pastries;Sandwiches' 'Coffee;Pastries;Sandwiches'
 'Coffee;Pastries' 'Cold drinks;Juices;Pastries'
 'Coffee;Cold drinks;Pastries;Sandwiches' 'Never' 'Never buy any'
 'Jaws chip ' 'cake ' 'Pastries' 'Cold drinks;Never' 'never' 'Nothing '
 'Coffee;Cold drinks;Juices;Pastries;Sandwiches']
10
['cake' 'coffee' 'cold drinks' 'jaws chip' 'juices' 'never'
 'never buy any' 'nothing' 'pastries' 'sandwiches']


,gender,age,job,income,visit_frequency,visit_method,visit_duration,distance,membership,item_list,...,purpose_biz,channel_list,revisit,item_cake,item_coffee,item_cold drinks,item_jaws chip,item_juices,item_pastries,item_sandwiches
0,Female,From 20 to 29,Student,"Less than RM25,000",Rarely,Dine in,Between 30 minutes to 1 hour,within 1km,Yes,coffee,...,3,Starbucks Website/Apps;Social Media;Emails;Dea...,Yes,0,1,0,0,0,0,0
1,Female,From 20 to 29,Student,"Less than RM25,000",Rarely,Take away,Below 30 minutes,1km - 3km,Yes,cold drinks;pastries,...,2,Social Media;In Store displays,Yes,0,0,1,0,0,1,0
2,Male,From 20 to 29,Employed,"Less than RM25,000",Monthly,Dine in,Between 30 minutes to 1 hour,more than 3km,Yes,coffee,...,3,In Store displays;Billboards,Yes,0,1,0,0,0,0,0
3,Female,From 20 to 29,Student,"Less than RM25,000",Rarely,Take away,Below 30 minutes,more than 3km,No,coffee,...,3,Through friends and word of mouth,No,0,1,0,0,0,0,0
4,Male,From 20 to 29,Student,"Less than RM25,000",Monthly,Take away,Between 30 minutes to 1 hour,1km - 3km,No,coffee;sandwiches,...,3,Starbucks Website/Apps;Social Media,Yes,0,1,0,0,0,0,1


In [59]:
df.columns

Index(['gender', 'age', 'job', 'income', 'visit_frequency', 'visit_method',
       'visit_duration', 'distance', 'membership', 'item_list',
       'visit_amount_spent', 'brand_rating', 'price_rating', 'promo_rating',
       'ambiance_rating', 'wifi_rating', 'service_rating', 'purpose_biz',
       'channel_list', 'revisit', 'item_cake', 'item_coffee',
       'item_cold drinks', 'item_jaws chip', 'item_juices', 'item_pastries',
       'item_sandwiches'],
      dtype='object')

In [60]:
## Similar to item_list, transform channel_list into a dataframe
# print(df['channel_list'].unique())

# check for missing values in channel_list
df['channel_list'].isna().sum()

# find the mode of channel_list
mode_channel = df['channel_list'].mode().values[0]
print(mode_channel)

# replace missing values in channel_list with the mode
df['channel_list'].fillna(mode_channel, inplace=True)

# check again
df['channel_list'].isna().sum()


## Perform the same process for promo_channel
vectorizer = CountVectorizer(tokenizer=lambda x: x.split(';'))

df['channel_list'] = df['channel_list'].str.strip().str.lower()
pr_channel = vectorizer.fit_transform(df['channel_list'])

print(len(vectorizer.get_feature_names_out()))
print(vectorizer.get_feature_names_out())

pr_channel_df = pd.DataFrame(pr_channel.toarray(), columns=vectorizer.get_feature_names_out())
pr_channel_df.columns = ['channel_' + col for col in pr_channel_df.columns]


df = pd.concat([df, pr_channel_df], axis=1)

Social Media
9
['application offer' 'billboards' 'deal sites (fave, iprice, etc...)'
 'emails' 'in store displays' 'never hear' 'social media'
 'starbucks website/apps' 'through friends and word of mouth']


In [61]:
df.columns

Index(['gender', 'age', 'job', 'income', 'visit_frequency', 'visit_method',
       'visit_duration', 'distance', 'membership', 'item_list',
       'visit_amount_spent', 'brand_rating', 'price_rating', 'promo_rating',
       'ambiance_rating', 'wifi_rating', 'service_rating', 'purpose_biz',
       'channel_list', 'revisit', 'item_cake', 'item_coffee',
       'item_cold drinks', 'item_jaws chip', 'item_juices', 'item_pastries',
       'item_sandwiches', 'channel_application offer', 'channel_billboards',
       'channel_deal sites (fave, iprice, etc...)', 'channel_emails',
       'channel_in store displays', 'channel_never hear',
       'channel_social media', 'channel_starbucks website/apps',
       'channel_through friends and word of mouth'],
      dtype='object')

In [62]:
df.drop(columns=['item_list', 'channel_list'], inplace=True)

In [63]:
df.info

<bound method DataFrame.info of      gender            age            job                income  \
0    Female  From 20 to 29        Student    Less than RM25,000   
1    Female  From 20 to 29        Student    Less than RM25,000   
2      Male  From 20 to 29       Employed    Less than RM25,000   
3    Female  From 20 to 29        Student    Less than RM25,000   
4      Male  From 20 to 29        Student    Less than RM25,000   
..      ...            ...            ...                   ...   
117    Male   40 and above  Self-employed   RM25,000 - RM50,000   
118    Male  From 20 to 29       Employed    Less than RM25,000   
119    Male  From 20 to 29        Student    Less than RM25,000   
120  Female  From 20 to 29       Employed    Less than RM25,000   
121    Male  From 20 to 29       Employed  RM50,000 - RM100,000   

    visit_frequency visit_method                visit_duration       distance  \
0            Rarely      Dine in  Between 30 minutes to 1 hour     within 1km   
1

In [66]:
# convert binary variables to numerical: gender, membership, purpose_biz, revisit
df['gender'] = df['gender'].map({'Male': 0, 'Female': 1})
df['membership'] = df['membership'].map({'No': 0, 'Yes': 1})
df['revisit'] = df['revisit'].map({'No': 0, 'Yes': 1})

In [82]:
df['age'] = df_copy['age']
df['income'] = df_copy['income']
df['visit_frequency'] = df_copy['visit_frequency']
df['visit_duration'] = df_copy['visit_duration']
df['distance'] = df_copy['distance']
df['visit_amount_spent'] = df_copy['visit_amount_spent']

# convert ordinal categorical variables to numerical: age, income, visit_frequency, visit_duration, distance, visit_amount_spent
df['age'].unique()
df['age'] = df['age'].map({ 'Below 20': 0, 'From 20 to 29': 1, 'From 30 to 39': 2, '40 and above': 3})
df['income'].unique()
df['income'] = df['income'].map({'Less than RM25,000': 0, 'RM25,000 - RM50,000': 1, 'RM50,000 - RM100,000': 2, 'RM100,000 - RM150,000': 3, 'More than RM150,000': 4})
df['visit_frequency'].unique()
df['visit_frequency'] = df['visit_frequency'].map({'Never': 0, 'Rarely': 1, 'Daily': 2, 'Weekly': 3, 'Monthly': 4})
df['visit_duration'].unique()
df['visit_duration'] = df['visit_duration'].map({'Below 30 minutes': 0, 'Between 30 minutes to 1 hour': 1, 'Between 1 hour to 2 hours': 2, 'Between 2 hours to 3 hours': 3, 'More than 3 hours': 4})
df['distance'].unique()
df['distance'] = df['distance'].map({'within 1km': 0, '1km - 3km': 1, 'more than 3km': 2})
df['visit_amount_spent'].unique()
df['visit_amount_spent'] = df['visit_amount_spent'].map({'Less than RM20': 0, 'Around RM20 - RM40': 1, 'More than RM40': 2})

In [64]:
# get the mean, medium for: brand_rating, price_rating, promo_rating, ambiance_rating, wifi_rating, service_rating, visit_amount_spent, purpose_biz
# keep the job and visit_method columns the same as there are no further analysis needed


In [86]:
df['visit_amount_spent'] = df_copy['visit_amount_spent']
df['visit_amount_spent'] = df['visit_amount_spent'].map({'Less than RM20': 0, 'Around RM20 - RM40': 1, 'More than RM40': 2})


In [87]:
df.head(10)

,gender,age,job,income,visit_frequency,visit_method,visit_duration,distance,membership,visit_amount_spent,...,item_sandwiches,channel_application offer,channel_billboards,"channel_deal sites (fave, iprice, etc...)",channel_emails,channel_in store displays,channel_never hear,channel_social media,channel_starbucks website/apps,channel_through friends and word of mouth
0,1,1,Student,0,1,Dine in,1,0,1,0.0,...,0,0,0,1,1,0,0,1,1,0
1,1,1,Student,0,1,Take away,0,1,1,0.0,...,0,0,0,0,0,1,0,1,0,0
2,0,1,Employed,0,4,Dine in,1,2,1,0.0,...,0,0,1,0,0,1,0,0,0,0
3,1,1,Student,0,1,Take away,0,2,0,0.0,...,0,0,0,0,0,0,0,0,0,1
4,0,1,Student,0,4,Take away,1,1,0,1.0,...,1,0,0,0,0,0,0,1,1,0
5,1,1,Student,0,1,Dine in,1,2,0,0.0,...,0,0,0,0,0,0,0,1,0,0
6,1,1,Student,0,1,Dine in,0,0,1,1.0,...,0,0,0,0,0,0,0,1,1,0
7,0,1,Employed,2,1,Dine in,1,2,1,0.0,...,0,0,0,0,1,0,0,1,1,1
8,1,1,Student,0,1,Drive-thru,0,2,1,2.0,...,0,0,0,0,0,0,0,1,1,1
9,0,1,Employed,0,4,Take away,0,2,0,1.0,...,0,0,0,0,0,0,0,1,0,1


In [89]:
df.to_csv('cleaned_data.csv', index=False)